In [1]:
#Import GYM stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

#Import helpers
import numpy as np
import random
import os
import pandas as pd

# Import stable baselines stuff
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
def get_data(start = '2017-01-01 00:00:00', end = '2017-12-12 23:55:00'):

    # import standard load profiles

    slp = pd.read_csv('df_p.csv', index_col=0, parse_dates=True)['0'] / 1000
    slp = slp.resample('15min').mean() * 3

    pv = pd.read_csv('Solar_Data-2011.csv', delimiter=';',
                    index_col=0, parse_dates=False)["Generation"] * 3
    pv.index = slp.index
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    
    return slp[start:end], pv[start:end]

In [9]:
class CostEnv(Env):
    def __init__(self):
        # Actions we can take increase in cost, lowering of cost
        self.action_space = Discrete(2)    
        ### Get input data, just choose one day for now
        self.load, self.pv = get_data(start = '2017-01-01 00:00:00', end = '2017-12-12 23:55:00')
        self.pv_price = 0.1
        self.grid_price = 0.4
        self.line_max = 15
        self.grid_penalty = 200
        self.battery_max = 20
        self.battery_state = 0
        self.pv_values = []
        self.grid_values = []
        ###
        
        # Set episode length
        self.episode_length = len(self.load)
        
        self.observation_space = Dict(
            {
                "load": Box(0, self.episode_length - 1, shape=(2,), dtype=int),
                "pv": Box(0, self.episode_length - 1, shape=(2,), dtype=int),
            }
        )
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 decrease in cost
        # 1 -1 = 0 Increase in cost
        # self.state += action -1 
        # Reduce episode length by 1 second
        # self.episode_length -= 1 ###Move to the end of the action
        ### We calculate the reward based on the price for the electricity, lower price, "higher" reward
        if action == 0:
            # Take all electricity from grid
            if self.load[len(self.load)-self.episode_length] > self.line_max:
                reward = self.load[len(self.load)-self.episode_length] * self.grid_price * -1 - abs(self.load[len(self.load)-self.episode_length] - self.line_max) * self.grid_penalty
            else:
                reward = self.load[len(self.load)-self.episode_length] * self.grid_price * -1
            
        elif action == 1:
            # Take all electricity from pv
            if self.pv[len(self.pv)-self.episode_length] >= self.load[len(self.load)-self.episode_length]:
                if abs(self.pv[len(self.pv)-self.episode_length] - self.load[len(self.load)-self.episode_length]) > self.line_max:
                    reward = self.load[len(self.load)-self.episode_length] * self.pv_price * -1 - (self.pv[len(self.pv)-self.episode_length] - self.load[len(self.load)-self.episode_length]) * self.grid_penalty
                else:
                    reward = self.load[len(self.load)-self.episode_length] * self.pv_price * -1
            else:
                if (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) > self.line_max:
                    reward = self.pv[len(self.pv)-self.episode_length] * self.pv_price * -1 - (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) * self.grid_price - (self.load[len(self.load)-self.episode_length] - self.pv[len(self.pv)-self.episode_length]) * self.grid_penalty
                else:
                    reward = self.pv[len(self.pv)-self.episode_length] * self.pv_price * -1
            ### This may lead the agent to always choose action 1, 
            ###because it will always supply the demand and will always be cheaper.
        
        else:
            # Invalid action
            raise(f'Invalid action: {action}')
        self.pv_values.append(self.pv[len(self.pv)-self.episode_length])
        self.grid_values.append(self.load[len(self.load)-self.episode_length])
        # Apply cost fluctuation
        # self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        ### Observation
        observation = {
            "load": self.load[len(self.load)-self.episode_length],
            "pv": self.pv[len(self.pv)-self.episode_length],
            }
        
        ### Either here or before checking self.episode_length
        self.episode_length -= 1
        ### Check if timeseries is over
        if self.episode_length <= 0: 
            done = True
        else:
            done = False
        
        # Return step information
        # return self.state, reward, done, info
        return observation, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        self.done=False
        # Set episode length
        self.episode_length = len(self.load)
        
        observation = {
            "load": (0, self.load[len(self.load)-self.episode_length]),
            "pv": (0, self.pv[len(self.pv)-self.episode_length]),
            }
        
        return observation





In [10]:
env = CostEnv()

In [13]:
episodes = 20
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        # n_state, reward, done, info = env.step(action)
        observation, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()


TypeError: step() missing 1 required positional argument: 'BaseException'

In [ ]:
log_path = os.path.join('Training', 'Logs')

model = A2C("MultiInputPolicy", env, verbose=1, tensorboard_log=log_path)

model.learn(total_timesteps=300000)

In [7]:
### Through this we evaluate our policy and observe we evaluation shall be proceeded with what level.
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=20)

C:\Users\mashood\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(-14334.387142150663, 0.0)

In [ ]:
model.save("A2C_Multi_Input_Policy_long_1")

In [ ]:
model = model.load("A2C_Multi_Input_Policy_long",env=env)

In [8]:
episodes=10

for ep in range(episodes):
    obs = env.reset()
    done = False
    while not done:
        action= model.predict(obs)
        obs,rewards,done,info,states = env.step(action)
        
env.close()

TypeError: exceptions must derive from BaseException

In [14]:
action_states = model.predict(obs)

In [15]:
action_states

(1, None)

In [11]:
obs,rewards,done,info = env.step(action)

TypeError: exceptions must derive from BaseException